# Lakeflow Jobs - Orkiestracja Workflows

---

## Kontekst biznesowy

**KION** potrzebuje zautomatyzować codzienny proces ETL:
- Ingestion danych zamówień o 2:00 w nocy
- Transformacja do warstwy Silver
- Agregacje do Gold
- Powiadomienia przy błędach

W tym module nauczysz się tworzyć i konfigurować Lakeflow Jobs przez UI.

---

## Agenda

1. Wprowadzenie do Lakeflow Jobs
2. Przygotowanie notebooków do Job (taski)
3. [DEMO UI] Tworzenie Multi-task Job
4. [DEMO UI] Triggery i Schedule
5. [DEMO UI] Opcje, Retry i Alerting
6. Praktyka: Widgets i parametry
7. Praktyka: Przekazywanie danych między taskami
8. Monitoring przez System Tables

---

## 1. Wprowadzenie do Lakeflow Jobs

**Lakeflow Jobs** (dawniej Databricks Jobs) to zarządzany serwis orkiestracji.

### Kiedy używać Jobs?

| Scenariusz | Rozwiązanie |
|------------|-------------|
| ETL pipeline z wieloma krokami | Multi-task Job |
| Codzienny raport o stałej godzinie | Scheduled Job |
| Reakcja na nowe pliki | File Arrival Trigger |
| ML training pipeline | Job z notebook taskami |
| Uruchomienie DLT pipeline | Job z DLT task |

### Jobs vs DLT (Lakeflow Pipelines)

| Cecha | Jobs | DLT |
|-------|------|-----|
| Orkiestracja | Ogólna | Tylko ETL |
| Zależności | Ręczna konfiguracja | Automatyczne (DAG) |
| Data Quality | Custom kod | Wbudowane expectations |
| Elastyczność | Wysoka | Opinionated |

**Best Practice**: Używaj DLT dla ETL pipelines, Jobs dla orkiestracji DLT + innych tasków.

---

## 2. Przygotowanie notebooków do Job

Poniżej znajdują się 3 proste notebooki, które użyjemy w demo.

**Instrukcja**: 
1. Utwórz folder `/Workspace/Users/<twoj-email>/jobs_demo/`
2. Skopiuj każdy z poniższych kodów do osobnego notebooka

---

### Task 1: Validate Source (`task_01_validate`)

In [0]:
# =============================================================================
# TASK 1: Validate Source Data
# Skopiuj ten kod do notebooka: task_01_validate
# =============================================================================

# Parametry z Job
dbutils.widgets.text("source_table", "samples.nyctaxi.trips")
dbutils.widgets.text("min_rows", "100")

source_table = dbutils.widgets.get("source_table")
min_rows = int(dbutils.widgets.get("min_rows"))

print(f"Validating: {source_table}")
print(f"Minimum rows required: {min_rows}")

# Walidacja
df = spark.table(source_table)
row_count = df.count()

if row_count < min_rows:
    raise Exception(f"Validation FAILED: {row_count} rows < {min_rows} minimum")

print(f"Validation PASSED: {row_count} rows")

# Zwróć wynik do następnego taska
import json
dbutils.notebook.exit(json.dumps({
    "status": "SUCCESS",
    "source_table": source_table,
    "row_count": row_count
}))

In [0]:
task_02_transform%md
### Task 2: Transform Data (`task_02_transform`)

In [0]:
# =============================================================================
# TASK 2: Transform Data
# Skopiuj ten kod do notebooka: task_02_transform
# =============================================================================

from pyspark.sql.functions import *
import json

# Parametry
dbutils.widgets.text("source_table", "samples.nyctaxi.trips")
dbutils.widgets.text("run_date", "")

source_table = dbutils.widgets.get("source_table")
run_date = dbutils.widgets.get("run_date") or str(current_date())

# Pobierz wynik z poprzedniego taska (opcjonalnie)
try:
    prev_result = dbutils.jobs.taskValues.get(
        taskKey="validate",
        key="returnValue",
        default="{}"
    )
    prev_data = json.loads(prev_result)
    print(f"Previous task result: {prev_data}")
except:
    print("Running standalone (no previous task)")

# Transformacja
print(f"Transforming: {source_table}")

df = spark.table(source_table)

df_transformed = (
    df
    .withColumn("trip_duration_minutes", 
                round((col("tpep_dropoff_datetime").cast("long") - 
                       col("tpep_pickup_datetime").cast("long")) / 60, 2))
    .withColumn("cost_per_mile", 
                when(col("trip_distance") > 0, 
                     round(col("fare_amount") / col("trip_distance"), 2))
                .otherwise(0))
    .withColumn("processing_date", lit(run_date))
)

row_count = df_transformed.count()
print(f"Transformed {row_count} rows")

# Wyświetl sample
df_transformed.select(
    "trip_distance", "fare_amount", "trip_duration_minutes", "cost_per_mile"
).show(5)

# Zwróć wynik
dbutils.notebook.exit(json.dumps({
    "status": "SUCCESS",
    "rows_transformed": row_count
}))

### Task 3: Generate Report (`task_03_report`)

In [0]:
# =============================================================================
# TASK 3: Generate Report
# Skopiuj ten kod do notebooka: task_03_report
# =============================================================================

from pyspark.sql.functions import *
import json
from datetime import datetime

# Parametry
dbutils.widgets.text("source_table", "samples.nyctaxi.trips")

source_table = dbutils.widgets.get("source_table")

print(f"Generating report from: {source_table}")

# Agregacje
df = spark.table(source_table)

report = df.agg(
    count("*").alias("total_trips"),
    round(sum("fare_amount"), 2).alias("total_revenue"),
    round(avg("fare_amount"), 2).alias("avg_fare"),
    round(avg("trip_distance"), 2).alias("avg_distance"),
    round(max("fare_amount"), 2).alias("max_fare")
).collect()[0]

# Wyświetl raport
print("\n" + "="*50)
print("DAILY REPORT")
print("="*50)
print(f"Total Trips:    {report.total_trips:,}")
print(f"Total Revenue:  ${report.total_revenue:,.2f}")
print(f"Avg Fare:       ${report.avg_fare:.2f}")
print(f"Avg Distance:   {report.avg_distance:.2f} miles")
print(f"Max Fare:       ${report.max_fare:.2f}")
print("="*50)
print(f"Generated at:   {datetime.now()}")
print("="*50 + "\n")

# Zwróć wynik
dbutils.notebook.exit(json.dumps({
    "status": "SUCCESS",
    "total_trips": report.total_trips,
    "total_revenue": float(report.total_revenue)
}))

---

## 3. [DEMO UI] Tworzenie Multi-task Job

### Checklist dla prowadzącego:

**Krok 1: Utwórz nowy Job**
- [ ] Workflows → Create Job
- [ ] Nazwa: `KION_Demo_ETL_Pipeline`

**Krok 2: Dodaj Task 1 (Validate)**
- [ ] Task name: `validate`
- [ ] Type: Notebook
- [ ] Path: `/Workspace/.../task_01_validate`
- [ ] Cluster: Serverless lub nowy Job Cluster
- [ ] Parameters: `source_table` = `samples.nyctaxi.trips`

**Krok 3: Dodaj Task 2 (Transform)**
- [ ] Task name: `transform`
- [ ] Depends on: `validate`
- [ ] Path: `/Workspace/.../task_02_transform`
- [ ] Parameters: `source_table` = `samples.nyctaxi.trips`

**Krok 4: Dodaj Task 3 (Report)**
- [ ] Task name: `report`
- [ ] Depends on: `transform`
- [ ] Path: `/Workspace/.../task_03_report`

**Krok 5: Uruchom Job**
- [ ] Run now
- [ ] Pokaż: DAG visualization
- [ ] Pokaż: Task logs i output

---

## 4. [DEMO UI] Triggery i Schedule

### Checklist dla prowadzącego:

**Opcje Triggerów** (zakładka Triggers):

| Trigger Type | Użycie | Przykład |
|--------------|--------|----------|
| **Scheduled** | Stały harmonogram | Codziennie o 2:00 |
| **File arrival** | Reakcja na nowe pliki | Nowy plik w `/landing/` |
| **Continuous** | Ciągłe przetwarzanie | Streaming-like |
| **Manual** | On-demand | Testowanie |

**Demo: Scheduled Trigger**
- [ ] Add trigger → Scheduled
- [ ] Cron expression: `0 0 2 * * ?` (codziennie 2:00)
- [ ] Timezone: `Europe/Warsaw`
- [ ] Pokaż: Preview next runs

**Demo: File Arrival Trigger** (opcjonalnie)
- [ ] Add trigger → File arrival
- [ ] URL: Unity Catalog Volume path
- [ ] Min files: 1
- [ ] Wait time: 5 minutes

### Przydatne wyrażenia CRON:

```
0 0 2 * * ?        # Codziennie o 2:00
0 0 * * * ?        # Co godzinę
0 0 9 ? * MON-FRI  # Pon-Pt o 9:00
0 0 0 1 * ?        # Pierwszy dzień miesiąca
0 */15 * * * ?     # Co 15 minut
```

---

## 5. [DEMO UI] Opcje, Retry i Alerting

### Checklist dla prowadzącego:

**Task-level options** (per task):
- [ ] Timeout: 30 minutes
- [ ] Retries: 2
- [ ] Retry delay: 60 seconds

**Job-level options** (górny panel):
- [ ] Max concurrent runs: 1 (zapobiega overlap)
- [ ] Job timeout: 2 hours

**Email Notifications**:
- [ ] On failure: `team@company.com`
- [ ] On success: (opcjonalnie)
- [ ] On start: (opcjonalnie)

**Webhook Integration** (Slack/Teams):
- [ ] Admin Settings → Destinations
- [ ] Add webhook URL
- [ ] Przypisz do Job

### Kiedy używać Retry?

| Scenariusz | Retry? | Dlaczego |
|------------|--------|----------|
| Network timeout | Tak | Transient error |
| API rate limit | Tak | Transient error |
| Data quality issue | Nie | Retry nie naprawi danych |
| Code bug | Nie | Retry nie naprawi kodu |

---

## 6. Praktyka: Widgets i Parametry

Databricks Widgets pozwalają parametryzować notebooki.

---

In [0]:
# Typy widgetów

# Text - dowolny tekst
dbutils.widgets.text("environment", "dev", "Environment")

# Dropdown - wybór z listy
dbutils.widgets.dropdown("region", "EU", ["EU", "US", "APAC"], "Region")

# Combobox - dropdown z możliwością wpisania
dbutils.widgets.combobox("table", "orders", ["orders", "customers", "products"], "Table")

# Multiselect - wielokrotny wybór
dbutils.widgets.multiselect("columns", "id", ["id", "name", "date", "amount"], "Columns")

In [0]:
# Pobieranie wartości
environment = dbutils.widgets.get("environment")
region = dbutils.widgets.get("region")
table = dbutils.widgets.get("table")
columns = dbutils.widgets.get("columns")  # zwraca string z przecinkami

print(f"Environment: {environment}")
print(f"Region: {region}")
print(f"Table: {table}")
print(f"Columns: {columns}")

In [0]:
# Dynamiczne parametry w Job
# Te wartości są dostępne gdy notebook jest uruchomiony jako task w Job

dynamic_params = {
    "{{job.start_time.iso_date}}": "Data uruchomienia (YYYY-MM-DD)",
    "{{job.start_time}}": "Pełny timestamp",
    "{{job.id}}": "ID Job",
    "{{run.id}}": "ID bieżącego uruchomienia",
    "{{task.name}}": "Nazwa bieżącego taska"
}

for param, description in dynamic_params.items():
    print(f"{param:35} -> {description}")

In [0]:
# Cleanup widgetów
dbutils.widgets.removeAll()

---

## 7. Praktyka: Przekazywanie danych między taskami

Dwa sposoby przekazywania danych:

1. **dbutils.notebook.exit()** - zwraca wartość z notebooka
2. **dbutils.jobs.taskValues** - odczytuje wartość z poprzedniego taska

---

In [0]:
# Task A - wysyła dane
import json

result = {
    "rows_processed": 1500,
    "max_date": "2024-01-15",
    "status": "SUCCESS"
}

# Zwróć jako JSON string
# dbutils.notebook.exit(json.dumps(result))
print(f"Task A would exit with: {json.dumps(result)}")

In [0]:
# Task B - odbiera dane z Task A
import json

# W rzeczywistym Job:
# task_a_output = dbutils.jobs.taskValues.get(
#     taskKey="task_a",           # nazwa poprzedniego taska
#     key="returnValue",          # klucz (domyślnie "returnValue")
#     default="{}",               # wartość domyślna
#     debugValue="{}"             # wartość do testowania lokalnie
# )

# Symulacja
task_a_output = '{"rows_processed": 1500, "max_date": "2024-01-15", "status": "SUCCESS"}'

data = json.loads(task_a_output)
print(f"Received from Task A:")
print(f"  Rows: {data['rows_processed']}")
print(f"  Max date: {data['max_date']}")
print(f"  Status: {data['status']}")

---

## 8. Monitoring przez System Tables

Databricks udostępnia system tables z historią Job runs.

---

In [0]:

spark.sql("""
    SELECT 
        *
    FROM system.lakeflow.job_run_timeline
    ORDER BY 1 DESC
    LIMIT 20
""").display()

In [0]:

spark.sql("""
    SELECT 
        DATE(period_start_time) as run_date,
        run_name as job_name,
        ROUND(
            AVG(
                (UNIX_TIMESTAMP(period_end_time) - UNIX_TIMESTAMP(period_start_time)) / 60
            ), 1
        ) as avg_duration_min,
        COUNT(*) as runs
    FROM system.lakeflow.job_run_timeline
    WHERE period_start_time >= current_date() - INTERVAL 14 DAYS
        AND result_state = 'SUCCESS'
    GROUP BY run_date, job_name
    ORDER BY run_date DESC
""").display()

---

## Podsumowanie

### W tym module nauczyłeś się:

| Temat | Kluczowe elementy |
|-------|-------------------|
| **Multi-task Jobs** | DAG workflow, zależności między taskami |
| **Triggery** | Scheduled (CRON), File arrival, Continuous |
| **Opcje** | Timeout, Retry, Max concurrent runs |
| **Alerting** | Email, Webhooks (Slack/Teams) |
| **Parametry** | Widgets, dynamic values, taskValues |
| **Monitoring** | System tables, success rate, duration trends |

### Best Practices:

1. **Serverless** - używaj dla większości Jobs (szybki start, auto-scaling)
2. **Idempotency** - Job powinien być bezpieczny do ponownego uruchomienia
3. **Retry** - tylko dla transient errors (network, API)
4. **Alerting** - zawsze konfiguruj powiadomienia o błędach
5. **Monitoring** - regularnie sprawdzaj success rate i trendy

### Następne kroki:

- **Notebook 04**: Unity Catalog Governance
- **Workshop**: Hands-on Job creation

---

## Materiały dodatkowe

- [Databricks Jobs Documentation](https://docs.databricks.com/workflows/jobs/jobs.html)
- [Serverless Jobs](https://docs.databricks.com/en/jobs/serverless.html)
- [Jobs Best Practices](https://docs.databricks.com/workflows/jobs/jobs-best-practices.html)

---